## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [1]:
from dotenv import load_dotenv

load_dotenv()
from fee_allocator.config_file import FEES_CONSTANTS
from fee_allocator.config_file import CORE_POOLS
import datetime
import json

import pandas as pd

from fee_allocator.helpers import get_block_by_ts
from fee_allocator.helpers import get_twap_bpt_price
from fee_allocator.config_file import Chains
from fee_allocator.config_file import WEB3_INSTANCES

# Load config file
with open('./protocol_fees_config.json') as f:
    CONFIG = json.load(f)['config']
REROUTE_CONFIG = CONFIG['reroute_config']
MIN_AURA_INCENTIVE = FEES_CONSTANTS['min_aura_incentive']

TARGET_BLOCKS = {}
timestamp_now = 1697148000
timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
datetime_2_weeks_ago = datetime.datetime.fromtimestamp(timestamp_2_weeks_ago)

target_blocks = {}
bpt_twap_prices = {chain.value: {} for chain in Chains}
# Collect BPT prices for 2 weeks ago and now for all relevant pools and chains
for chain in Chains:
    target_blocks[chain.value] = (
        get_block_by_ts(timestamp_now, chain.value),  # Block now
        get_block_by_ts(timestamp_2_weeks_ago, chain.value)  # Block 2 weeks ago
    )
    print(f"Collecting bpt prices for {chain.value}")
    pools = CORE_POOLS.get(chain.value, None)
    if pools is None:
        continue
    for core_pool in pools:
        bpt_twap_prices[chain.value][core_pool] = get_twap_bpt_price(
            core_pool, chain.value, getattr(WEB3_INSTANCES, chain.value),
            start_date=datetime.datetime.fromtimestamp(timestamp_now), block_number=target_blocks[chain.value][0]
        )

## Fetching fees and other data from the Balancer subgraphs

In [3]:
from fee_allocator.config_file import BALANCER_GRAPH_URLS
from fee_allocator.helpers import get_balancer_pool_snapshots
from typing import Dict

pool_snapshots = {}
for chain in Chains:
    pool_snapshots[chain.value] = (
        get_balancer_pool_snapshots(target_blocks[chain.value][0], BALANCER_GRAPH_URLS[chain.value]),  # now
        get_balancer_pool_snapshots(target_blocks[chain.value][1], BALANCER_GRAPH_URLS[chain.value]),  # 2 weeks ago
    )

## Extract fee data for CORE pools:


In [4]:
from collections import defaultdict
from decimal import Decimal
from fee_allocator.helpers import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: Chains, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.datetime) -> Dict[str, Dict]:
    """
    Collects fee info for all pools in the list. Returns dictionary with pool id as key and fee info as value 
    """
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # If pools doesn't have current fees it means it was not created yet, so we skip it
        if not current_fees_snapshots:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        token_fees_in_usd = 0
        bpt_price_usd = bpt_twap_prices[chain.value][pool] or 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'] or 0)  # If 2 weeks ago is null, set to 0
        else:
            # Collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None) or 0)
                # Get twap token price from Balancer API
                token_price = fetch_token_price_balgql(token_data['address'], chain.value, start_date) or 0
                token_fees_in_usd += Decimal(token_fee) * Decimal(token_price)
        fees[pool_snapshot_now['pool']['id']] = {
            'symbol': pool_snapshot_now['pool']['symbol'],
            'pool_addr': pool_snapshot_now['pool']['address'],
            'bpt_token_fee': round(bpt_token_fee, 2),
            # One of two fields below should always be 0 because fees are taken in either BPT or pool tokens
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(token_fees_in_usd, 2),
            'chain': chain.value,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']]
        }
    return fees


collected_fees = {}

for chain in Chains:
    core_pools = CORE_POOLS.get(chain.value, None)
    if not core_pools:
        continue
    collected_fees[chain.value] = collect_fee_info(core_pools, chain, pool_snapshots[chain.value][0],
                                                   pool_snapshots[chain.value][1], datetime_now)
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**collected_fees[Chains.MAINNET.value], **collected_fees[Chains.ARBITRUM.value],
              **collected_fees[Chains.POLYGON.value], **collected_fees[Chains.BASE.value],
              **collected_fees[Chains.AVALANCHE.value], **collected_fees.get(Chains.GNOSIS.value, {})}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

## Now let's calculate incentives paid to the pools and save them to csv

In [5]:
from fee_allocator.helpers import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(WEB3_INSTANCES.mainnet, target_blocks[Chains.MAINNET.value][0])

# Incentives are split per chain and per pool, with each pool getting a share of the incentive proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the incentives. That 10% will be distributed between aura and vebal, proportional to their share of the incentive.
# TODO: This should be automated somehow or live in config file
FEES_TO_DISTRIBUTE = {
    Chains.MAINNET.value: Decimal(84_904),
    Chains.ARBITRUM.value: Decimal(60_986.399974),
    Chains.POLYGON.value: Decimal(6_032.493545),
    Chains.BASE.value: Decimal(5_848.9779),
    Chains.GNOSIS.value: Decimal(8_311.6239),
    Chains.AVALANCHE.value: Decimal(10_241.454),
}
SUM = sum(FEES_TO_DISTRIBUTE.values())
print(SUM)
# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_incentives(fees: Dict, chain: str, fees_to_distribute: Decimal) -> Dict[str, Dict]:
    pool_incentives = {}
    # Calculate pool share in fees
    dao_share = Decimal(FEES_CONSTANTS['dao_share_pct'])
    vebal_share = Decimal(FEES_CONSTANTS['vebal_share_pct'])
    fees_to_distr_wo_dao_vebal = fees_to_distribute - (fees_to_distribute * dao_share) - (
            fees_to_distribute * vebal_share)
    # Calculate totals
    bpt_fees = sum([data['bpt_token_fee_in_usd'] for pool, data in fees.items()])
    token_fees = sum([data['token_fees_in_usd'] for pool, data in fees.items()])
    total_fees = bpt_fees + token_fees
    if not total_fees:
        return {}
    aura_bal_switch = True
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        pool_share = pool_fees / Decimal(total_fees)
        # If aura incentives is less than 500 USDC, we pay all incentives to balancer
        aura_incentives = round(pool_share * fees_to_distr_wo_dao_vebal * aura_vebal_share, 2)
        if aura_incentives <= Decimal(MIN_AURA_INCENTIVE):
            if aura_bal_switch:
                aura_incentives = Decimal(0)
                bal_incentives = round(pool_share * fees_to_distr_wo_dao_vebal, 2)
                aura_bal_switch = not aura_bal_switch
            else:
                aura_incentives = round(pool_share * fees_to_distr_wo_dao_vebal, 2)
                bal_incentives = Decimal(0)
                aura_bal_switch = not aura_bal_switch

        else:
            bal_incentives = round(pool_share * fees_to_distr_wo_dao_vebal * (1 - aura_vebal_share), 2)
        fees_to_dao = round(pool_share * fees_to_distribute * dao_share, 2)
        fees_to_vebal = round(pool_share * fees_to_distribute * vebal_share, 2)
        # Split fees between aura and bal fees
        pool_incentives[pool] = {
            "chain": chain,
            "symbol": data['symbol'],
            "protocol_fees_collected": pool_fees,
            "fees_to_vebal": fees_to_vebal,
            "fees_to_dao": fees_to_dao,
            "total_incentives": aura_incentives + bal_incentives,
            "aura_incentives": aura_incentives,
            "bal_incentives": bal_incentives,
            "redirected_incentives": Decimal(0),
            "reroute_incentives": Decimal(0),
            "pool_addr": data['pool_addr'],
        }
    return pool_incentives


def re_route_incentives(incentives_data: Dict[str, Dict], chain: Chains) -> Dict[str, Dict]:
    """
    If pool is in re-route configuration, all incentives from that pool should be distributed to destination pool
      Ex: {source_pool: destination_pool}
    """
    if not chain.value in REROUTE_CONFIG:
        return incentives_data
    for pool_id, _data in incentives_data.items():
        if pool_id in REROUTE_CONFIG[chain.value]:
            # Re route everything to destination pool and set source pool incentives to 0
            incentives_data[REROUTE_CONFIG[chain.value][pool_id]]['aura_incentives'] += _data['aura_incentives']
            incentives_data[REROUTE_CONFIG[chain.value][pool_id]]['bal_incentives'] += _data['bal_incentives']
            # Increase total incentives by aura and bal incentives
            incentives_data[REROUTE_CONFIG[chain.value][pool_id]]['total_incentives'] += _data['aura_incentives'] + \
                                                                                         _data[
                                                                                             'bal_incentives']
            # Mark source pool incentives as rerouted
            incentives_data[REROUTE_CONFIG[chain.value][pool_id]]['reroute_incentives'] += _data['total_incentives']
            incentives_data[pool_id]['aura_incentives'] = 0
            incentives_data[pool_id]['bal_incentives'] = 0
    return incentives_data


def re_distribute_incentives(incentives_data: Dict[str, Dict], chain: Chains) -> Dict[str, Dict]:
    """
    If some pools received < min_vote_incentive_amount all incentives from that pool 
        should be evenly distributed between all pools that received > min_vote_incentive_amount
    """
    # Collect pools that received < min_vote_incentive_amount
    pools_to_redistribute = {}
    for pool_id, _data in incentives_data.items():
        if _data['total_incentives'] < Decimal(FEES_CONSTANTS['min_vote_incentive_amount']):
            pools_to_redistribute[pool_id] = _data
    # Collect pools that received > min_vote_incentive_amount
    pools_to_receive = {}
    for pool_id, _data in incentives_data.items():
        if _data['total_incentives'] > Decimal(FEES_CONSTANTS['min_vote_incentive_amount']):
            pools_to_receive[pool_id] = _data
    # Redistribute incentives
    for pool_id, _data in pools_to_redistribute.items():
        # Calculate incentives to redistribute
        incentives_to_redistribute = _data['total_incentives']
        incentives_to_redistribute_aura = _data['aura_incentives']
        incentives_to_redistribute_bal = _data['bal_incentives']
        # Calculate incentives to receive
        incentives_to_receive = round(incentives_to_redistribute / len(pools_to_receive), 2)
        incentives_to_receive_aura = round(incentives_to_redistribute_aura / len(pools_to_receive), 2)
        incentives_to_receive_bal = round(incentives_to_redistribute_bal / len(pools_to_receive), 2)
        # Set incentives to redistribute to 0
        incentives_data[pool_id]['total_incentives'] = 0
        incentives_data[pool_id]['aura_incentives'] = 0
        incentives_data[pool_id]['bal_incentives'] = 0
        # Mark incentives as redistributed
        incentives_data[pool_id]['redirected_incentives'] = -incentives_to_redistribute
        # Redistribute incentives
        for pool_id_to_receive, _data_to_receive in pools_to_receive.items():
            incentives_data[pool_id_to_receive]['total_incentives'] += incentives_to_receive
            incentives_data[pool_id_to_receive]['redirected_incentives'] += incentives_to_receive
            incentives_data[pool_id_to_receive]['aura_incentives'] += incentives_to_receive_aura
            incentives_data[pool_id_to_receive]['bal_incentives'] += incentives_to_receive_bal
    return incentives_data


incentives = {}
for chain in Chains:
    if not collected_fees.get(chain.value):
        continue
    _incentives = calc_and_split_incentives(collected_fees[chain.value], chain.value,
                                            FEES_TO_DISTRIBUTE[chain.value])
    # Reroute incentives if needed
    re_routed_incentives = re_route_incentives(_incentives, chain)
    incentives[chain.value] = re_distribute_incentives(re_routed_incentives, chain)

joint_incentives_data = {**incentives[Chains.MAINNET.value], **incentives[Chains.ARBITRUM.value],
                         **incentives[Chains.POLYGON.value], **incentives[Chains.BASE.value],
                         **incentives[Chains.AVALANCHE.value]}

joint_incentives_data = {k: v for k, v in sorted(joint_incentives_data.items(), key=lambda item: item[1]['chain'])}
joint_incentives_df = pd.DataFrame.from_dict(joint_incentives_data, orient='index')
# Don't show pool_addr
display = joint_incentives_df.drop(columns=['pool_addr'])
display_sorted = display.sort_values(by=['chain', 'protocol_fees_collected'], ascending=False)
display_sorted.to_csv(f'../allocations/incentives_{datetime_2_weeks_ago.date()}_{datetime_now.date()}.csv')
display_sorted

176324.9493190000002869055607


,chain,symbol,protocol_fees_collected,fees_to_vebal,fees_to_dao,total_incentives,aura_incentives,bal_incentives,redirected_incentives,reroute_incentives
0xf0ad209e2e969eaaa8c882aac71f02d8a047d5c2000200000000000000000b49,polygon,ECLP-WMATIC-stMATIC,1941.79,1017.75,548.02,1734.08,624.25,1109.83,168.31,0
0xcd78a20c597e367a4e478a2411ceb790604d7c8f000000000000000000000c22,polygon,maticX-WMATIC-BPT,1381.35,724.01,389.85,1282.17,0.10,1282.07,168.31,0
0xee278d943584dd8640eaf4cc6c7a5c80c0073e85000200000000000000000bc7,polygon,ECLP-WMATIC-MATICX,417.21,218.67,117.75,0,0,0,-336.42,0
0x89b753153678bc434c610b7e9182297cada8ff29000000000000000000000c21,polygon,stMATIC-WMATIC-BPT,0.25,0.13,0.07,0,0,0,-0.20,0
0xdc31233e09f3bf5bfe5c10da2014677c23b6894c000000000000000000000c23,polygon,wstETH-WETH-BPT,0.00,0.00,0.00,0,0,0,0.00,0
0x42ed016f826165c2e5976fe5bc3df540c5ad0af700000000000000000000058b,mainnet,wstETH-rETH-sfrxETH-BPT,16250.38,5908.16,3181.32,9113.87,3624.01,5489.86,24.39,0
0x93d199263632a4ef4bb438f1feb99e57b4b5f0bd0000000000000000000005c2,mainnet,wstETH-WETH-BPT,15578.32,5663.82,3049.75,8737.96,3474.16,5263.80,24.39,0
0x1e19cf2d73a72ef1332c882f20534b6519be0276000200000000000000000112,mainnet,B-rETH-STABLE,14747.17,5361.64,2887.04,8273.07,3288.85,4984.22,24.39,0
0x1ee442b5326009bb18f2f472d3e0061513d1a0ff000200000000000000000464,mainnet,50rETH-50BADGER,5688.56,2068.19,1113.64,3206.22,1269.08,1937.14,24.39,0
0x37b18b10ce5635a84834b26095a0ae5639dcb7520000000000000000000005cb,mainnet,ETHx-WETH-BPT,5550.99,2018.18,1086.71,3129.28,1238.41,1890.87,24.39,0
